In [ ]:
import torch

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import warnings 

warnings.filterwarnings('ignore')

# Load and transform data

**1) ladda**

Vi kommer att använda följande dataset:

https://archive.ics.uci.edu/dataset/320/student+performance

Vårt mål kommer vara att förutspå varje elevs slutbetyg, beroende på olika faktorer.
I hemsidan kan du läsa på mer detaljerat om vad varje kolumn beskriver!

In [ ]:
students_df = pd.read_csv('../data/students.csv', delimiter=';')

**2) Initial data check**

Vi vill predikta 'G3' (slutbetyg) givet övriga attribut.

Slutbetyget anges med ett värde på 0-20, där 20 är bästa betyg.

In [ ]:
students_df

In [ ]:
print(students_df.G3.value_counts())

students_df.G3.hist(bins=30);

Som syns ovan ser vi att ingen fått maxbetyg (20) men att två elever uppnåde 19 iaf. De allra flesta verkar fått slutbetyg mellan 9 till 15.

**3) välj bort vissa kolumner**

In [ ]:
#vi väljer bort ett par kolumner för enkelhetens skull, och tar endast med ett par utvalda

target_column = ['G3']
feature_columns = ['school', 'sex', 'age', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'reason',
                   'studytime', 'failures', 'higher', 'internet', 'Dalc', 'Walc', 'health', 'G1', 'G2']

students_df = students_df[feature_columns + target_column]

students_df

**4) transformera data**

In [ ]:
students_df.info()

Alla inputs och outputs för neurala nätverk måste först siffror (integer eller floats). Vi ser att flera av våra kolumner (bl.a. *school* och *sex*) inte är det, och behöver således åtgärda det. Ett av kolumnerna är dessutom kategorisk, men vi återkommer till den strax.

Först och främst ser vi att ett antal kolumner (*school*, *sex*, *famsize*, *pstatus*, *higher* och *internet*) är binära - dvs att de bara antar två värden. Dessa kan vi helt enkelt omvandla dessa två värden till 1 och 0, respektive. 

**Kontrollera själv att kolumnerna ovan verkligen är binära**

In [ ]:
binary_columns = ['school', 'sex', 'famsize', 'Pstatus', 'higher', 'internet']

for column in binary_columns:
    
    first_value = students_df[column].unique()[0] # extrahera ett av de binära värdena
    transformed_column = [1 if value == first_value else 0 for value in students_df[column]]
    
    students_df[column] = transformed_column

In [ ]:
#Kontrollera nu att alla värden omvandlats till 1 och 0 i de binära kolumnerna

for column in binary_columns:
    print(students_df[column].value_counts(), end='\n\n')

Nu återstår att åtgärda den kategoriska kolumnen *reason*.

Vi behöver göra om den här kolumnen till siffror, och en strategi för att hantera kategoriska kolumner är att omvandla dem 
till kolumner, en varje varje värde - och på enklaste sätt ange 1 eller 0 för respektive kolumn på de rader som värdet antas.

Detta kallas också 'one-hot-encoding'.

In [ ]:
# För varje möjlig kategoriskt värde, loopa och konstruera en ny kolumn enligt ovan

categorical_columns = ['reason']

for column in categorical_columns:
    for value in set(students_df[column].values):
    
        onehotencode = [1 if x == value else 0 for x in students_df[column]]
        students_df[value] = onehotencode
    

#slutligen, droppa orginalkolumnen som vi inte längre behöver    

for column in categorical_columns:
    students_df = students_df.drop(columns=[column])

In [ ]:
students_df

Nu ser det redan bättre ut!

Låt oss bara re-arrange så att slutbetyg-kolumnen (G3) är sista kolumnen i dataframet. Det blir lite lättare då.

In [ ]:
g3 = students_df.pop('G3') #droppa columnen från students_df, och fånga upp den i variablen g3
students_df['G3'] = g3     #lägg tillbaks kolumnen. På detta sätt hamnar den på sista plats

Nu går vi vidare!

**5) skala data**

In [ ]:
students_df.info()

Vi ser nu att samtliga kolumner är av rätt datatyp.

Det som återstår är att *normalisera* **input** kolumnerna. Detta är **superviktigt** att göra när man tränar neurala nätverk, för att ingen enstaka kolumn ska dominera de övriga i storlek. I vårt fall är det inte så illa, eftersom att värdena bland alla input kolumner är mellan 0 och som högst 20. 

Men, vi normaliserar iaf - det är best practice.

Detta kan göras på olika sätt, men vanligtvis innebär detta att man skalar om värdena i respektive kolumn till att vara mellan [0,1] eller [-1,1]. Det spelar egentligen inte särskilt stor roll vilken av dessa skalor man väljer, men jag brukar välja [0,1] för kolumner som bara ha positiva värden, och [-1,1] för kolumner som har både positiva och negativa värden.

Eftersom att alla våra inputkolumner endast antar positiva värden, 

kan vi således försöka skala de till [0,1]. Ett enkelt knep för att åstadkomma detta är helt enkelt att dela varje kolumn på sitt högsta värde.

OBS: Vi normaliserar **inte** vår target kolumn.

In [ ]:
feature_columns = students_df.columns[:-1]

for feature in feature_columns:
    
    students_df[feature] = students_df[feature]/max(students_df[feature].values) #dela varje inputkolumn på sitt högsta värde
    

In [ ]:
students_df

Sista steget är att omvandla datan till ett format som är optimalt för PyTorch. Ett format som kallas för *tensor*.

In [ ]:
training_set = torch.tensor(students_df.values).float()            #nu är allt klart, så vi anger detta som vår training_set

#säkerställ att datasetet är av samma storlek

print(students_df.shape)

print(training_set.shape)

In [ ]:
training_set           # visualisera hela tensor-dataset

In [ ]:
training_set[:,-1]     # precis som vanligt kan vi ex få sista kolumnen (vilket motsvarar vår target 
                       # slutbetyg i detta fall) genom denna query

**Sådär, all done. Nu kan vi gå vidare!**

# Skapa Neurons med PyTorch

Att skapa modeller med PyTorch görs allra oftast via klasser. Lyckligtvis är detta supersimpelt.

In [ ]:
import torch.nn as nn

class neuron(nn.Module):
    def __init__(self, input_size):
        super(neuron, self).__init__()
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        x = self.fc(x)
        return x

Vår input size till modellen är ju alla våra features, och de har vi 20 st av. **Eller hur?!**

In [ ]:
input_size = 20

model = neuron(input_size) # initiera en instans av vår neuron-klass

Vi kan se en summary av vår modell genom att kalla på den

In [ ]:
model

Om vi vill kan vi direkt räkna ut antal parametrar genom följande kodsnutt

**Fråga: varför är antalet parametrar som det är?**

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {total_params}")

Vi kan läsa av dessa parametrar direkt via model.parameters(), där de är lagrade

Lägg märke till att sifforna är helt random inom intervallet [-1,1]. 

**För att kontrollera detta, testa att initiera om modellen ovan, och dra följande kodsnutt igen**

In [ ]:
weights, bias = model.parameters()

print('vikter')
print(weights, end='\n\n')

print('bias')
print(bias, end='\n\n')

För att ge input till vår modell, och få en output gör man helt enkelt såhär:

In [ ]:
sample_student = training_set[0,:-1]      # ta den första studentens input features (första raden)
sample_grade = training_set[0,-1]         # extrahera även den studentens slutbetyg

model_prediction = model(sample_student)  # predicta ett slutbetyg med vår färskt initierade modell

In [ ]:
print('True grade            :', sample_grade.item())
print('Vår models predict    :', model_prediction.item())


Ha, katastrofalt fel! 

Sifforna är inte ens i närheten av nära, eller hur? :) 

Det är OK, för vi har inte börjat träna.

Men nu är dags!

## Träna med PyTorch

Vi väljer först en loss function. Eftersom att vi kör regression kan vi exempelvis välja Mean Absolute Error (MAE) loss - även mer tekniskt kallat för L1 Loss. 

Kom ihåg att det här kommer användas för att kvantifier avståndet mellan våra prediktions och det sanna värdet. Vi kommer alltså vilja minimera denna loss.

In [ ]:
loss_function = torch.nn.L1Loss()

Vi väljer också en så kallad optimizer. Det är den här som kommer utföra själva gradient descent steget vi pratat om.

Notera här att vi också lägger in en parameter kallad *lr*. Detta är learning rate vi också pratat om, och bestämmer hur stort steg varje gradient descent tar när den uppdaterar våra parametrar.

In [ ]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr = 0.001) # observera att vi här visar vår optimizer vilka modellens parametrar är, 
                                                # så att den vet vad ska uppdatera 

Nu under träningens gång så kommer inte vi skicka in hela datasetet på en gång, utan vi skickar in ett par training samples åt gången. Antalet samples vi skickar in per iteration kallas för *batch size*. Varför varje sådan batch kommer vi att utföra gradient descent och uppdatera (träna) våra parametrar. Mer om detta längre ner.

Vanliga batchsizes är typ mellan 16-128. Vi kan gott köra med 16.

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_set,                  # det är denna funktion som kommer ansvara för att leverera
                              batch_size = 16,               # samples till modellen under träningens gång
                              shuffle=True)

**Lite mer om dataloader och bathsize:**

När vi väljer batch_size = 16 så kommer den, för varje gång den blir kallad, att välja randomly 16 stycken training samples som den levererar.

Vi kan också se hur en sådan här batch ser ut, samt den första training samplen ur den batch

In [ ]:
for batch in train_dataloader:
    
    print(batch.size(), end='\n\n') # visa storleken för hela den här batchen
    
    print(batch[0,-1])              # printa första training samples slutbetyg, i den här batchen
    print(batch[0,:-1])             # printa alla feature kolumns för första training sample i den här batchen
    
    break

Vi ser att det är totalt 16 rader (training samples) i batchen, och 21 kolumner. Den sista kolumnen i varje rad motsvarar slutbetyg, precis som tidigare - och övriga är våra input features.

Hur många batches har dataloader till oss?

In [ ]:
len(train_dataloader)

Vi kan även, för tydlighetens skull, se antalet training samples i varenda sådan batch

In [ ]:
batch_nummer = 0

for batch in train_dataloader:
    
    batch_nummer += 1

    print(f'Batch: {batch_nummer}.  ',f'Antal samples: {len(batch)}')

Om vi summerar antalet samples som levererades totalt av våra 41 batches ovan blir det exakt 649 - vilket är storleken på vårt dataset. DataLoader kommer alltså leverera (efter en komplett for-loop) lika manga training samples som storleken på vårt dataset!

Som vi ser försöker den ge 16 samples för varje batch, förutom den sista – eftersom att vi skulle överskrida storleken på vårt dataset. Räkna ihop antalet ovan, och jämför med len(training_set).

**Viktigt begrepp**:

En komplett for-loop genom DataLoader kallas för en **epoch**. 

Dvs, när vår modell fått träna på lika manga samples som storleken på vart dataset, så sager vi att modellen fått träna i en epoch.

**Nu kör vi hela träningsloopen**

In [ ]:
# ändra inget av det här, förens du blir instruerad till det

input_size = 20                       
batch_size = 16

epochs = 100               # antal loopar genom dataloader vi låter vår modell träna på vårt dataset.
learning_rate = 0.001      # hur stora steg gradient descent tar.

Som ni ser i träningskoden nedan är värdet på epochs antal gånger som vi kommer gå igenom datan i vår train_dataloader. 

Eftersom att vår train_dataloader innehar 41 batches, så kommer vi totalt att låta algoritmen gå igenom (41 x epochs) batches.

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
#    initera modell, loss_function, optimizer & dataloader


model = neuron(input_size)           
loss_function = torch.nn.L1Loss() 
optimizer = SGD(model.parameters(), lr = learning_rate)

train_dataloader = DataLoader(training_set,                 
                              batch_size = batch_size,       
                              shuffle=True)

# ----------------------------------------------------------------------------------------------------------------
#    träna


model.train()

batch_losses = []

for i in range(epochs):

    for batch in train_dataloader:
    
        y_true = batch[:,-1]
        input_features = batch[:,:-1]

        y_pred = model(input_features)
        loss = loss_function(y_true, y_pred)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        batch_loss = loss.item()
        batch_losses.append(batch_loss)
        
        print(np.round(batch_loss,3))

Vi kan nu plotta loss-historiken

In [ ]:
plt.plot(batch_losses);
plt.xlabel('batch');
plt.ylabel('training loss (MAE)');

Losset som beräknades under den allra sista iteration av träning är

In [ ]:
batch_losses[-1]

Detta betyder att *medelfelet* av våra prediktions, på hela vårt träningssätt är det värdet du ser ovan! Men betygen för varje person är ju också satt mellan 0-20, så ett medelfel på värdet du ser ovan är inte helt katastrofalt dåligt.

Det är ju dock ändå knappast optimalt - och anledningen är för att en Neuron är en ganska dålig modell :)

Här ville vi ju dock bara lära oss grunderna i PyTorch. Vi kommer lära oss betydligt bättre kraftfullare modeller snart.

## UPPGIFTER

**0)**

Kolla på plotten över lossen ovan. Hur tolkar du den? 

**1)** 

Låt oss direkt testa hur den tränade modellen presterar på vårt dataset. Välj själv en training sample, ange ett värde mellan 0-648.

Kan du hitta några training samples som modellen predictar särskilt bra/dåligt på? Jämför felen du får i dina svar med medelfelet vi fick ovan.

In [ ]:
model.eval();                                        # sätt modellen i evaluerings/predicion läge

training_sample = 100                                 # TESTA OLIKA VÄRDEN PÅ DENNA

y_true = training_set[training_sample, -1]           # extrahera sanna slutbetyget för givet training sample
input_features = training_set[training_sample, :-1]  # extrahera alla input features för samma training sample

y_prediction = model(input_features)                 # predicta slutbetyg, givet input features


print('True grade       :', y_true.item())
print('Predicted grade  :', y_prediction.item())

**2)**

Åskådliggör modellens parametrar på nytt. 

Säkerställ att de har ändrats sedan vi initierade modellen.

In [ ]:
weights, bias = 

**3)**

Gå nu tillbaks till träningsloopen och testa att köra igenom den för ett par olika värden på epochs, och plotta på nytt. 

Testa värden på epochs med början på 1 och sluta på 50. Exempelvis 1, 5, 10, 15, ..., 45, 50.

Vad verkar hända?

**4)**

Återigen, gå tillbaks till träningsloopen. Sätt epochs = 100. Testa nu istället olika värden på learning rate

Testa följande värden på learning rate, i tur ordning

- 0.00001 
- 0.01
- 1

och plotta på nytt vår loss. Vad verkar hända? 

*Hint: jämför lossen* 